# Import packages nécessaires

In [19]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
from tqdm import tqdm
import re
import time

# Fonction permettant de récupérer tous les magasins de la caisse

### 14/10/2019: 1,370 magasins sur toute la caisse (fonction marche)

In [75]:
def get_links_from_department():
    
    # URL du site web à scraper
    std_url = 'https://www.au-magasin.fr/'
    
    # Listes regroupant les départements par caisse (ou au total => variable "ALL")
    CATP = ['86-vienne/', '37-indre-et-loire/']
    CACL = ['18-cher/', '45-loiret/', '58-nievre/']
    CACO = ['36-indre/', '87-haute-vienne/']
    CAVF = ['28-eure-et-loir/', '41-loir-et-cher/']
    ALL = ['86-vienne/', '37-indre-et-loire/', '18-cher/', '45-loiret/', '58-nievre/', '36-indre/', '87-haute-vienne/', '28-eure-et-loir/', '41-loir-et-cher/']
    
    # Liste vide dans laquelle nous ajouterons les liens des différentes villes
    links = []
    
    # Boucle permettant d'itérer sur la liste de départements choisi dans le but de récupérer les url de toutes les villes en
    # faisant parti
    for i in tqdm(CAVF):
        url = std_url + i
        response = get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        for link in soup.findAll('a', attrs={'href': re.compile("^(https:\/\/www.au-magasin.fr\/[0-9]{5}-)([a-z-]+\/$)")}):
            links.append(link.get('href'))
            
    print(len(links))
    
    return links

In [76]:
get_links_from_department()

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.42it/s]


293


['https://www.au-magasin.fr/28410-abondant/',
 'https://www.au-magasin.fr/28500-allainville/',
 'https://www.au-magasin.fr/28800-alluyes/',
 'https://www.au-magasin.fr/28300-amilly/',
 'https://www.au-magasin.fr/28260-anet/',
 'https://www.au-magasin.fr/28290-arrou/',
 'https://www.au-magasin.fr/28700-aunay-sous-auneau/',
 'https://www.au-magasin.fr/28500-aunay-sous-crecy/',
 'https://www.au-magasin.fr/28700-auneau/',
 'https://www.au-magasin.fr/28330-authon-du-perche/',
 'https://www.au-magasin.fr/28320-bailleau-armenonville/',
 'https://www.au-magasin.fr/28300-bailleau-l-eveque/',
 'https://www.au-magasin.fr/28120-bailleau-le-pin/',
 'https://www.au-magasin.fr/28630-barjouville/',
 'https://www.au-magasin.fr/28310-baudreville/',
 'https://www.au-magasin.fr/28140-bazoches-les-hautes/',
 'https://www.au-magasin.fr/28480-beaumont-les-autels/',
 'https://www.au-magasin.fr/28240-belhomert-guehouville/',
 'https://www.au-magasin.fr/28630-bercheres-les-pierres/',
 'https://www.au-magasin.fr

# Fonction permettant de récupérer les informations souhaitées pour une ville donnée

### 14/10/2019: (pour la ville de Poitiers) 566 magasins trouvés. Informations collectées: nom, adresse, ville, catégorie. Informations stockées dans le fichier excel "POITIERS.xls"

In [77]:
def ScrapAuMagasinOnePage():
    
    # Calculer le temps d'exécution du script
    start = time.time()
    
    cities_url = get_links_from_department()
    #is_there_next_page = True
    
    # Liste des informations à recueillir
    names = []
    addresses = []
    cities = []
    postal_code = []
    categories = []

    
    for link in cities_url:
        
        # Récupérer un lien et regarder s'il y a une pagination
        url = link
        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        header = html_soup.find('div', 'paginationBlock d-flex col-12')
        
        # Si FALSE => fonction pour récupérer les informations d'une seule page
        if header != None:
            print(header is not None)
            html_soup_one_page = BeautifulSoup(response.text, 'html.parser')
            store_containers_one_page = html_soup_one_page.find_all('div', 'shop')
                       
            page = 0
            #page += 1

            is_there_next_page = True

            #while page != 1:
            while is_there_next_page == True:
                try:
                    page += 1
                    url = link + str(page)
                    print(url)

                    response = get(url)
                    print(response)
                    # Lancer une session BeautifulSoup & Localiser/Stocker l'adresse de localisation du contenu de la page recherché
                    html_soup = BeautifulSoup(response.text, 'html.parser')
                    store_containers = html_soup.find_all('div', 'shop')

                    # Emplacement du bouton "next page"
                    header = html_soup.find('div', 'paginationBlock d-flex col-12')
                    print(header is None)
                    next_button = header.find('li', 'reverse right disabled')


                    # Exctration des données pour chaque block de la page
                    for i in tqdm(store_containers):

                        # Condition de vérification que la page soit toujours valide et agencée de la même manière
                        if i.find('div', 'font-weight-light') is not None:

                            # Récupérer les noms
                            name = i.a.h2.text
                            name = name.strip()
                            names.append(name)

                            # Récupérer les adresses
                            address = i.find('div', 'font-weight-light').text
                            address = address.strip()
                            addresses.append(address)

                            # Extraire le nom de la ville dans une variable séparée
                            city = address.split()[-1]
                            cities.append(city)
                            
                            # Extraire le code postal dans une variable séparée
                            p_code = address.split()[-2]
                            postal_code.append(p_code)

                            # Récupérer les catégories
                            category = i.li.text
                            categories.append(category)
                        
                    if next_button:
                        is_there_next_page = False

                except Exception as ex:
                    print(ex)
                    print("Probablement dernière page: ", page)
                    break
                    
        else:
            url = link
            print(url)
            response = get(url)
            html_soup = BeautifulSoup(response.text, 'html.parser')
            store_containers = html_soup.find_all('div', 'shop')
            
            for i in tqdm(store_containers):
                # Condition de vérification que la page soit toujours valide et agencée de la même manière
                if i.find('div', 'font-weight-light') is not None:

                    # Récupérer les noms
                    name = i.a.h2.text
                    name = name.strip()
                    names.append(name)

                    # Récupérer les adresses
                    address = i.find('div', 'font-weight-light').text
                    address = address.strip()
                    addresses.append(address)

                    # Extraire le nom de la ville dans un variable séparée
                    city = address.split()[-1]
                    cities.append(city)
                    
                    # Extraire le code postal dans une variable séparée
                    p_code = address.split()[-2]
                    postal_code.append(p_code)

                    # Récupérer les catégories
                    category = i.li.text
                    categories.append(category)
            


    # Regrouper les 3 variables (names, addresses et categories) dans un numpy array (= liste de liste dans ce cas)
    datas = np.column_stack((names, addresses, cities, postal_code, categories))
    print("Résultat variable datas {}".format(datas))
    
    execution_time = (time.time() - start)/60
    print('Le script a pris {} minutes pour s\'exécuter.'.format(execution_time))
    # Créer un DataFrame Pandas avec les informations de datas
    return pd.DataFrame(data=datas, columns=['Nom_magasin', 'Adresse', 'Ville', 'Code_postal', 'Categorie'])

# Lancement du scrap pour les 4 caisses

### En fonction du résultat souhaité, il faut modifier la fonction get_links_from_department
##### Si on désire obtenir un résultat par caisse => remplacer la boucle "for" => tqdm(nom_de_la_caisse_souhaité)
##### Si on désire obtenir un résultat total => remplacer la boucle "for" => tqdm(ALL)

#### Temps estimé: 8 minutes pour CATP et CACL, 5 minutes pour CACO et CAVF

In [27]:
df_CATPT = ScrapAuMagasinOnePage()
print(df_CATP)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.37it/s]


373
https://www.au-magasin.fr/86430-adriers/


100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


https://www.au-magasin.fr/86260-angles-sur-l-anglin/


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 1505.31it/s]


https://www.au-magasin.fr/37290-yzeures-sur-creuse/


100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1250.87it/s]


Résultat variable datas [['La Poste Adriers' '43 Rue Principale - 86430 Adriers' 'Adriers'
  '86430' 'Service postal']
 ['Vival Adriers' '1 Rue Des Tilleuls - 86430 Adriers' 'Adriers' '86430'
  'Supermarché']
 ["La Poste Angles-sur-l'Anglin"
  "11 Rue De Tournon - 86260 Angles-sur-l'Anglin" "Angles-sur-l'Anglin"
  '86260' 'Service postal']
 ...
 ['Aréas Yzeures-sur-Creuse' 'Les Chalussons - 37290 Yzeures-sur-Creuse'
  'Yzeures-sur-Creuse' '37290' 'Banques / Assurances']
 ['Best Mountain Yzeures-sur-Creuse'
  '59 Rue Pasteur - 37290 Yzeures-sur-Creuse' 'Yzeures-sur-Creuse'
  '37290' 'Magasin de vêtement']
 ['Square Habitat Yzeures-sur-Creuse'
  '2 Place Mado Robin - 37290 Yzeures-sur-Creuse' 'Yzeures-sur-Creuse'
  '37290' 'Agences immobilières']]
Le script a pris 444.13432812690735 secondes pour s'exécuter.


NameError: name 'df_CATP' is not defined

In [62]:
df_CACL = ScrapAuMagasinOnePage()
print(df_CACL)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.04s/it]


435
https://www.au-magasin.fr/18250-acheres/


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


https://www.au-magasin.fr/18200-ainay-le-vieil/


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.14it/s]


https://www.au-magasin.fr/58370-villapourcon/


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 267.53it/s]


Résultat variable datas [["Tape à l'oeil Achères"
  '10, Rue Des Communes\nCc Leclerc - 18250 Achères' 'Achères' '18250'
  'Magasin bébé / Enfant']
 ['La Poste Ainay-le-Vieil' 'Place De L Eglise - 18200 Ainay-le-Vieil'
  'Ainay-le-Vieil' '18200' 'Service postal']
 ['La Poste Allogny' 'Place De La Mairie - 18110 Allogny' 'Allogny'
  '18110' 'Service postal']
 ...
 ['Pharmacie Granier Pascale Varzy' '2 Faubourg Marcy - 58210 Varzy'
  'Varzy' '58210' 'Pharmacie']
 ['La Poste Verneuil' '- 58300 Verneuil' 'Verneuil' '58300'
  'Service postal']
 ['La Poste Villapourçon' '- 58370 Villapourçon' 'Villapourçon' '58370'
  'Service postal']]
Le script a pris 7.729076143105825 secondes pour s'exécuter.
                                            Nom_magasin  \
0                                 Tape à l'oeil Achères   
1                               La Poste Ainay-le-Vieil   
2                                      La Poste Allogny   
3                  domaine du gué de la férolle Arcomps   
4     

[4690 rows x 5 columns]


In [71]:
df_CACO = ScrapAuMagasinOnePage()
print(df_CACO)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.42it/s]


269
https://www.au-magasin.fr/36140-aigurande/


100%|████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 2496.83it/s]


https://www.au-magasin.fr/36120-ambrault/


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 478.09it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


https://www.au-magasin.fr/87260-vicq-sur-breuilh/


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 500.24it/s]


Résultat variable datas [['Intermarché Aigurande' 'La Combette - 36140 Aigurande' 'Aigurande'
  '36140' 'Supermarché']
 ['Pulsat Aigurande' '32 Place De La Promenade - 36140 Aigurande'
  'Aigurande' '36140' 'Magasin hifi vidéo']
 ['Top Garage Aigurande' '11, Avenue De Verdun - 36140 Aigurande'
  'Aigurande' '36140' 'Garagiste']
 ...
 ['La Poste Veyrac' '- 87520 Veyrac' 'Veyrac' '87520' 'Service postal']
 ['La Poste Vicq-sur-Breuilh' 'Le Bourg - 87260 Vicq-sur-Breuilh'
  'Vicq-sur-Breuilh' '87260' 'Service postal']
 ['Gamm vert Vicq-sur-Breuilh'
  'Le Pont De Geasse - 87260 Vicq-sur-Breuilh' 'Vicq-sur-Breuilh' '87260'
  'Jardinerie']]
Le script a pris 5.041917622089386 minutes pour s'exécuter.
                                        Nom_magasin  \
0                             Intermarché Aigurande   
1                                  Pulsat Aigurande   
2                              Top Garage Aigurande   
3              Pharmacie Lefranc Nouaille Aigurande   
4                      

[2464 rows x 5 columns]


In [78]:
df_CAVF = ScrapAuMagasinOnePage()
print(df_CAVF)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.35it/s]


293
https://www.au-magasin.fr/28410-abondant/


100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


https://www.au-magasin.fr/28500-allainville/


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


https://www.au-magasin.fr/41600-yvoy-le-marron/


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Résultat variable datas [['carrefour Abondant' 'Lieu Dit Le Boutoin - 28410 Abondant' 'Abondant'
  '28410' 'Supermarché']
 ['La Poste Abondant' '20 Grande Rue - 28410 Abondant' 'Abondant' '28410'
  'Service postal']
 ['Auto Primo Allainville' '2 Route De Dreux - 28500 Allainville'
  'Allainville' '28500' 'Magasin auto']
 ...
 ['Pharmacie Ulrich Daniel Vouzon' '36 Grande Rue - 41600 Vouzon'
  'Vouzon' '41600' 'Pharmacie']
 ['La Poste Vouzon' '17 La Grande Rue - 41600 Vouzon' 'Vouzon' '41600'
  'Service postal']
 ['La Poste Yvoy-le-Marron' '9 Grande Rue - 41600 Yvoy-le-Marron'
  'Yvoy-le-Marron' '41600' 'Service postal']]
Le script a pris 5.328553978602091 minutes pour s'exécuter.
                               Nom_magasin  \
0                       carrefour Abondant   
1                        La Poste Abondant   
2                   Auto Primo Allainville   
3                         La Poste Alluyes   
4                             Solea Amilly   
5            Pharmacie Henri-Barbot 

[2845 rows x 5 columns]


# CATP - Data processing

In [32]:
df_CATPT.head()

,Nom_magasin,Adresse,Ville,Code_postal,Categorie
0,La Poste Adriers,43 Rue Principale - 86430 Adriers,Adriers,86430,Service postal
1,Vival Adriers,1 Rue Des Tilleuls - 86430 Adriers,Adriers,86430,Supermarché
2,La Poste Angles-sur-l'Anglin,11 Rue De Tournon - 86260 Angles-sur-l'Anglin,Angles-sur-l'Anglin,86260,Service postal
3,La Poste Angliers,10 Allee Louis Aubert De Tourny - 86330 Angliers,Angliers,86330,Service postal
4,Archambault Fils Marée Crustacés Angliers,1 Impasse Du Quebec - 86330 Angliers,Angliers,86330,Supermarché


In [34]:
df_CATPT.describe()

,Nom_magasin,Adresse,Ville,Code_postal,Categorie
count,3992,3992,3992,3992,3992
unique,3502,3805,372,91,80
top,La Poste Tours,Centre Commercial Les Atlantes - 37700 Saint-P...,Tours,37000,Banques / Assurances
freq,19,13,814,814,449


In [35]:
categories = df_CATPT['Categorie'].unique()
print(len(categories))
print(categories)

80
['Service postal' 'Supermarché' 'Boulangerie / Pâtisserie' 'Magasin auto'
 'Magasin bricolage' 'Banques / Assurances' 'Garagiste' 'Pharmacie'
 'Discount' 'Presse / Tabac' 'Opticiens / Audition'
 'Plomberie / Sanitaire' 'Chauffage et climatisation' 'Salles de sport'
 'Service aux particuliers' 'Porte et fenêtre' 'Magasin électroménager'
 'Fournitures / Professionnels' 'Magasin de meuble' 'Magasin loisirs'
 'Magasin chaussures' 'Magasin jeux vidéo' 'Magasin de vêtement'
 'Occasions / Revente' 'Magasin de jouet' 'Magasin téléphone'
 'Magasin déco' 'Cuisine' 'Restauration rapide' 'Esthétique'
 'Magasin de sport' 'Magasin bijoux et accessoires' 'Animalerie'
 'Coiffeurs' 'Boucherie / Charcuterie / Traiteur' 'Transport'
 'Piscine / Spa' 'Restaurants' 'Magasin bio' 'Magasin bébé / Enfant'
 'Loisirs' 'Hôtel / Hébergement' 'Couverture / Toiture'
 'Construction de maison' 'Station-service / Lavage'
 'Confiserie / Chocolaterie' 'Jardinerie' 'Magasin informatique'
 'Agences de voyages' 'Déménage

# Indiquer les magasins faisant partis d'un centre commercial

In [42]:
zone_commerciale = [
    'CC', 'Cc', 'Centre Commercial', 'C. Cial', 'C Cial', 'Zone Commercial', 'Cc.', 'C/c', 'C.c.', 'C.C.', 'Centre Cial',
    'Galerie Marchande', 'Leclerc', 'Galerie Commerciale', 'Ctre Cial', 'Auchan', 'Zac', 'zac', 'ZAC', 'Parc Commercial',
    'Parc commercial', 'Espace Commercial', 'Espace commercial'
]

zone_industrielle = ['Zi', 'Zi.','Z.i', 'Z.i', 'Z.i.', 'Za', 'Za.','Z.a', 'Z.a', 'Z.a.', 'Zone Industrielle', 'Zone']

autre = ['Aire', 'Autoroute']

In [50]:
catp = df_CATPT.copy()
catp.head()

,Nom_magasin,Adresse,Ville,Code_postal,Categorie
0,La Poste Adriers,43 Rue Principale - 86430 Adriers,Adriers,86430,Service postal
1,Vival Adriers,1 Rue Des Tilleuls - 86430 Adriers,Adriers,86430,Supermarché
2,La Poste Angles-sur-l'Anglin,11 Rue De Tournon - 86260 Angles-sur-l'Anglin,Angles-sur-l'Anglin,86260,Service postal
3,La Poste Angliers,10 Allee Louis Aubert De Tourny - 86330 Angliers,Angliers,86330,Service postal
4,Archambault Fils Marée Crustacés Angliers,1 Impasse Du Quebec - 86330 Angliers,Angliers,86330,Supermarché


In [56]:
catp['zone_commerciale'] = catp['Adresse'].str.contains('CC|Cc|Centre Commercial|C. Cial|C Cial|Zone Commercial|Cc.|C/c|C.c.|C.C.|Centre Cial|Galerie Marchande|Galerie Commerciale|Ctre Cial|Auchan|Leclerc|Zac|zac|ZAC|Parc Commercial|Parc commercial|Espace Commercial|Espace commercial', regex=True)*1

catp['zone_industrielle'] = catp['Adresse'].str.contains('Zi|Zi.|Z.i|Z.i|Z.i.|Za|Za.|Z.a|Z.a|Z.a.|Zone Industrielle|Zone', regex=True)*1

catp['autre'] = catp['Adresse'].str.contains('Aire|Autoroute', regex=True)*1

In [57]:
catp.head(150)

,Nom_magasin,Adresse,Ville,Code_postal,Categorie,zone_commerciale,zone_industrielle,autre
0,La Poste Adriers,43 Rue Principale - 86430 Adriers,Adriers,86430,Service postal,0,0,0
1,Vival Adriers,1 Rue Des Tilleuls - 86430 Adriers,Adriers,86430,Supermarché,0,0,0
2,La Poste Angles-sur-l'Anglin,11 Rue De Tournon - 86260 Angles-sur-l'Anglin,Angles-sur-l'Anglin,86260,Service postal,0,0,0
3,La Poste Angliers,10 Allee Louis Aubert De Tourny - 86330 Angliers,Angliers,86330,Service postal,0,0,0
4,Archambault Fils Marée Crustacés Angliers,1 Impasse Du Quebec - 86330 Angliers,Angliers,86330,Supermarché,0,0,0
5,Archambault Yannick Angliers,18 Allee Aubert De Tourny - 86330 Angliers,Angliers,86330,Boulangerie / Pâtisserie,0,0,0
6,Auto Primo Angliers,9 Route Nationale - 86330 Angliers,Angliers,86330,Magasin auto,0,0,0
7,La Poste Antigny,3 Place De La Mairie - 86310 Antigny,Antigny,86310,Service postal,0,0,0
8,Brioche Dorée Antran,Aire De Chatellerault-Usseau - 86100 Antran,Antran,86100,Boulangerie / Pâtisserie,0,0,1
9,La Poste Arçay,Route Chaseigne - 86200 Arçay,Arçay,86200,Service postal,0,0,0


In [58]:
catp.to_excel("dataset_catp.xls", index=False)

# CACL - Data processing

In [65]:
cacl = df_CACL.copy()
cacl.head()

,Nom_magasin,Adresse,Ville,Code_postal,Categorie
0,Tape à l'oeil Achères,"10, Rue Des Communes\nCc Leclerc - 18250 Achères",Achères,18250,Magasin bébé / Enfant
1,La Poste Ainay-le-Vieil,Place De L Eglise - 18200 Ainay-le-Vieil,Ainay-le-Vieil,18200,Service postal
2,La Poste Allogny,Place De La Mairie - 18110 Allogny,Allogny,18110,Service postal
3,domaine du gué de la férolle Arcomps,Les Frêles - 18200 Arcomps,Arcomps,18200,Service animalier
4,La Poste Arcomps,Fosse Nouvelle - 18200 Arcomps,Arcomps,18200,Service postal


In [66]:
cacl['zone_commerciale'] = cacl['Adresse'].str.contains('CC|Cc|Centre Commercial|C. Cial|C Cial|Zone Commercial|Cc.|C/c|C.c.|C.C.|Centre Cial|Galerie Marchande|Galerie Commerciale|Ctre Cial|Auchan|Leclerc|Zac|zac|ZAC|Parc Commercial|Parc commercial|Espace Commercial|Espace commercial', regex=True)*1

cacl['zone_industrielle'] = cacl['Adresse'].str.contains('Zi|Zi.|Z.i|Z.i|Z.i.|Za|Za.|Z.a|Z.a|Z.a.|Zone Industrielle|Zone', regex=True)*1

cacl['autre'] = cacl['Adresse'].str.contains('Aire|Autoroute', regex=True)*1

In [67]:
cacl.to_excel("dataset_cacl.xls", index=False)

# CACL - Data processing

In [72]:
caco = df_CACO.copy()
caco.head()

,Nom_magasin,Adresse,Ville,Code_postal,Categorie
0,Intermarché Aigurande,La Combette - 36140 Aigurande,Aigurande,36140,Supermarché
1,Pulsat Aigurande,32 Place De La Promenade - 36140 Aigurande,Aigurande,36140,Magasin hifi vidéo
2,Top Garage Aigurande,"11, Avenue De Verdun - 36140 Aigurande",Aigurande,36140,Garagiste
3,Pharmacie Lefranc Nouaille Aigurande,26 Place Champ De Foire - 36140 Aigurande,Aigurande,36140,Pharmacie
4,La Poste Aigurande,55 Place Du Champ De Foire - 36140 Aigurande,Aigurande,36140,Service postal


In [73]:
caco['zone_commerciale'] = caco['Adresse'].str.contains('CC|Cc|Centre Commercial|C. Cial|C Cial|Zone Commercial|Cc.|C/c|C.c.|C.C.|Centre Cial|Galerie Marchande|Galerie Commerciale|Ctre Cial|Auchan|Leclerc|Zac|zac|ZAC|Parc Commercial|Parc commercial|Espace Commercial|Espace commercial', regex=True)*1

caco['zone_industrielle'] = caco['Adresse'].str.contains('Zi|Zi.|Z.i|Z.i|Z.i.|Za|Za.|Z.a|Z.a|Z.a.|Zone Industrielle|Zone', regex=True)*1

caco['autre'] = caco['Adresse'].str.contains('Aire|Autoroute', regex=True)*1

In [74]:
caco.to_excel("dataset_caco.xls", index=False)

# CAVF - Data processing

In [79]:
cavf = df_CAVF.copy()
cavf.head()

,Nom_magasin,Adresse,Ville,Code_postal,Categorie
0,carrefour Abondant,Lieu Dit Le Boutoin - 28410 Abondant,Abondant,28410,Supermarché
1,La Poste Abondant,20 Grande Rue - 28410 Abondant,Abondant,28410,Service postal
2,Auto Primo Allainville,2 Route De Dreux - 28500 Allainville,Allainville,28500,Magasin auto
3,La Poste Alluyes,9 Place Du Donjon - 28800 Alluyes,Alluyes,28800,Service postal
4,Solea Amilly,Rn23 - 28300 Amilly,Amilly,28300,Magasin déco


In [80]:
cavf['zone_commerciale'] = cavf['Adresse'].str.contains('CC|Cc|Centre Commercial|C. Cial|C Cial|Zone Commercial|Cc.|C/c|C.c.|C.C.|Centre Cial|Galerie Marchande|Galerie Commerciale|Ctre Cial|Auchan|Leclerc|Zac|zac|ZAC|Parc Commercial|Parc commercial|Espace Commercial|Espace commercial', regex=True)*1

cavf['zone_industrielle'] = cavf['Adresse'].str.contains('Zi|Zi.|Z.i|Z.i|Z.i.|Za|Za.|Z.a|Z.a|Z.a.|Zone Industrielle|Zone', regex=True)*1

cavf['autre'] = cavf['Adresse'].str.contains('Aire|Autoroute', regex=True)*1

In [81]:
cavf.to_excel("dataset_cavf.xls", index=False)